In [1]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
import portfolio
import dataAck
import warnings
warnings.filterwarnings("ignore")
import portfolioGeneration
from IPython.display import display

In [2]:
modelHashes = portfolioGeneration.getAllPortfolioModels()

In [3]:
allModels = portfolio.getModelsByKey(modelHashes)

In [4]:
joinedData = portfolioGeneration.getPertinentDataForModels(allModels)

((10, ('XLV', 31, None, 14), 2, 'VO'), 10, 50)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)
((10, ('IEF', 3, None, None), 2, 'EWT'), 10, 50)
((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((22, ('TIP', 29, None, None), 3, 'OEF'), 22, 25)
((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25)
((22, ('IWF', 4, None, None), 3, 'VO'), 22, 25)
((22, ('XLV', 26, None, None), 3, 'OEF'), 22, 25)
((10, ('EWH', 5, None, 10), 2, 'EWT'), 10, 25)
GDX
TIP
VO
SHY
IEF
OEF
EWZ
EWT
XLV
IWF
EWH


In [5]:
##SHOULD BE NAN IN MANY SERIES...DON'T WANT TO UNNECESSARILY TRUNCATE
display(joinedData)

,Adj_Close_VO,Adj_Close_XLV,Adj_Close_EWZ,Adj_Close_GDX,Adj_Close_SHY,Adj_Close_EWT,Adj_Close_IEF,Adj_Close_OEF,Adj_Close_TIP,Adj_Close_IWF,Adj_Close_EWH
Date,,,,,,,,,,,
1996-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.896262
1996-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.054796
1996-04-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.019567
1996-04-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.896262
1996-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.834609
1996-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.936581
1996-04-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.861032
1996-04-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.861032
1996-04-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.861032


In [9]:
portfolioGeneration.generateRawPredictions(allModels, joinedData, daysBack = True)

((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25) -1.0 2017-08-14 00:00:00
((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25) -1.0 2017-08-11 00:00:00
((10, ('EWH', 5, None, 10), 2, 'EWT'), 10, 25) 1.0 2017-08-14 00:00:00
((10, ('EWH', 5, None, 10), 2, 'EWT'), 10, 25) 1.0 2017-08-11 00:00:00
((22, ('TIP', 29, None, None), 3, 'OEF'), 22, 25) -1.0 2017-08-14 00:00:00
((22, ('TIP', 29, None, None), 3, 'OEF'), 22, 25) -1.0 2017-08-10 00:00:00
((22, ('TIP', 29, None, None), 3, 'OEF'), 22, 25) 1.0 2017-08-11 00:00:00
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50) 1.0 2017-08-14 00:00:00
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50) 1.0 2017-08-10 00:00:00
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50) -1.0 2017-08-11 00:00:00
((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25) 1.0 2017-08-14 00:00:00
((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25) 1.0 2017-08-11 00:00:00
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25) 1.0 2017-08-14 00:00:00
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)

In [17]:
##UPLOAD MODEL PREDICTIONS

In [18]:
##STORE TODAY AGGREGATE FOR ALL MODELS
def produceModelPredictions(allModels, joinedData):
    for model in allModels:
        todayAggregatePrediction = portfolio.getAggregatePredictionForModelDaily(model, joinedData)
        print(model.describe(), todayAggregatePrediction)
        if portfolio.getToday() == joinedData.index[-1]:
            print("SKIPPING UPLOAD, MUST WAIT UNTIL TOMORROW")
            continue
        
        portfolio.storeAggregateModelPrediction(model,\
                            todayAggregatePrediction, portfolio.getToday(), shouldReturn = False)

In [19]:
produceModelPredictions(allModels, joinedData)

((10, ('TIP', 24, None, 15), 2, 'VO'), 10, 25) -1.0
((10, ('EWH', 5, None, 10), 2, 'EWT'), 10, 25) 1.0
((22, ('TIP', 29, None, None), 3, 'OEF'), 22, 25) -0.3333333333333333
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50) 0.3333333333333333
((10, ('SHY', 30, None, None), 2, 'VO'), 10, 25) 1.0
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25) 1.0
((22, ('XLV', 26, None, None), 3, 'OEF'), 22, 25) 1.0
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100) 1.0
((10, ('XLV', 31, None, 14), 2, 'VO'), 10, 50) 0.0
((10, ('IEF', 3, None, None), 2, 'EWT'), 10, 50) 0.0
((22, ('IWF', 4, None, None), 3, 'VO'), 22, 25) 0.3333333333333333


In [13]:
##UPLOAD PORTFOLIO PREDICTIONS

In [6]:
allPortfolios = portfolio.getPortfolios()

In [7]:
modelsInPortfolio = {}
for portfolioInfo in allPortfolios:
    print(portfolioInfo)
    models = portfolio.getModelsByKey(portfolio.getPortfolioModels(portfolioInfo["key"]))
    modelsInPortfolio[portfolioInfo["key"]] = models
    
    for model in models:
        print(model.describe())

{'key': '1e0c232022c226ff39f67cc3e0686593237394f1989b2c08e76d0a21', 'description': 'First Combo Trade Test vs SPY', 'benchmark': 'SPY'}
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((22, ('TIP', 29, None, None), 3, 'OEF'), 22, 25)
((22, ('XLV', 26, None, None), 3, 'OEF'), 22, 25)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)
{'key': '5522786eb28943c781c8bf62503848a2bdf51ba50e612fd13c5a3f9f', 'description': 'Just trade underlying', 'benchmark': 'EWZ'}
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
{'key': '6258238755a1b74f251d1455565b494a05fec30eba348b8a17101c42', 'description': 'Just trade underlying vs SPY', 'benchmark': 'SPY'}
((10, ('IEF', 3, None, None), 2, 'EWT'), 10, 50)
((10, ('EWH', 5, None, 10), 2, 'EWT'), 10, 25)
{'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f', 'description': 'VO 8-13-17', 'benchmark': 'VO'}
((10, ('SHY', 30, None, None), 3, 'VO'), 10, 25)
((22, ('IWF', 4, None, None), 3, 'VO')

In [16]:
print(allModels)

[<dataAck.algoBlob object at 0x119826240>, <dataAck.algoBlob object at 0x119826048>, <dataAck.algoBlob object at 0x1198269b0>, <dataAck.algoBlob object at 0x119826828>, <dataAck.algoBlob object at 0x1198266a0>, <dataAck.algoBlob object at 0x119826470>, <dataAck.algoBlob object at 0x1198269e8>, <dataAck.algoBlob object at 0x119826e48>, <dataAck.algoBlob object at 0x119826cf8>, <dataAck.algoBlob object at 0x119826d68>, <dataAck.algoBlob object at 0x119826ba8>]


In [20]:
aggregateReturns, aggregatePredictions = portfolioGeneration.generateAggregateReturnsPredictions(allModels, joinedData)

In [21]:
display(aggregatePredictions)

,341fe3e8eb88c9fbdd779c6c8727762f99fc1043383dd8242da044e9,a762825ed2a3e080b702b24d23121d77b9e6b6fe24af40f5d3187f44,736666aec4fadbeac073674a18f5702bd69156f73b8649077cb59c05,e1111d514a667ab22d8f48a2734a7dab0be1350f8ce70ca3b8a172ac,2c69141675447782e2a74036a7cbd8fb5fc08476cebf557bd6e787a6,0b84734d97e005d3a2a3a06d82cb793ec1f7557573e3bc032fb7fdf3,4018e7b6f84216af0c0b942847d8ee439f42a184fcc9a0ccea90f212,bafabbb2ddbb6e294ed821e1e1e41116981d49addb415b95798bb67e,78afec28d6c4e55102b09d8b6b179a3508e2b03fb46d2e8858c14fdc,b903708f9b21d044698db01d68f6be915739a688f28c591eb57807c3,11912f289e86b30f1af1b7272e773732dd099e3069cdf79f1c2ba47e
2006-11-06,0.0,1.0,NaN,NaN,0.0,1.0,NaN,NaN,-0.5,0.0,-0.500000
2006-11-07,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,-1.0,0.0,-0.500000
2006-11-08,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,-1.0,-1.0,0.000000
2006-11-09,1.0,0.0,NaN,NaN,1.0,0.5,NaN,NaN,-1.0,0.0,0.000000
2006-11-10,1.0,-1.0,NaN,NaN,0.0,0.0,NaN,NaN,-1.0,1.0,-0.500000
2006-11-13,1.0,0.0,NaN,NaN,0.0,-0.5,NaN,NaN,0.0,1.0,-0.500000
2006-11-14,0.0,0.0,NaN,NaN,0.0,-1.0,NaN,NaN,0.5,0.0,-1.000000
2006-11-15,-1.0,0.0,NaN,NaN,-1.0,-1.0,NaN,NaN,0.5,-1.0,-1.000000
2006-11-16,0.0,0.0,NaN,NaN,-1.0,-1.0,NaN,NaN,0.0,0.0,-0.500000
2006-11-17,0.0,-1.0,NaN,NaN,0.0,-1.0,NaN,NaN,-0.5,1.0,0.000000


In [23]:
##GENERATE WEIGHTS FOR PORTFOLIO TODAY
allocationsToStore = []
for portfolioKey in modelsInPortfolio:
    hrpReturns, historicalWeights = portfolioGeneration.produceHRPPredictions(aggregateReturns[[portfolio.getModelHash(model) for model in modelsInPortfolio[portfolioKey]]], 22, startIndex=3, maxWindowSize=True)
    print(portfolioKey, historicalWeights.iloc[-1])
    todayWeight = historicalWeights.iloc[-1]
    netPosition = {}
    transformedAlgoPrediction = {}
    for model in modelsInPortfolio[portfolioKey]:
        if model.inputSeries.targetTicker not in netPosition:
            netPosition[model.inputSeries.targetTicker] = 0.0
        netPosition[model.inputSeries.targetTicker] += todayWeight[portfolio.getModelHash(model)] * portfolio.getAggregatePredictionForModelDaily(model, joinedData)
        transformedAlgoPrediction[portfolio.getModelHash(model)] = todayWeight[portfolio.getModelHash(model)] * portfolio.getAggregatePredictionForModelDaily(model, joinedData)
    print(portfolioKey, netPosition)
    allocationsToStore.append(portfolioGeneration.storePortfolioAllocation(portfolioKey, portfolio.getToday(), todayWeight.to_dict(), netPosition, transformedAlgoPrediction, shouldReturn=True))
portfolio.storeManyItems(allocationsToStore)

1e0c232022c226ff39f67cc3e0686593237394f1989b2c08e76d0a21 4018e7b6f84216af0c0b942847d8ee439f42a184fcc9a0ccea90f212    0.557748
736666aec4fadbeac073674a18f5702bd69156f73b8649077cb59c05    0.378065
bafabbb2ddbb6e294ed821e1e1e41116981d49addb415b95798bb67e    0.031763
e1111d514a667ab22d8f48a2734a7dab0be1350f8ce70ca3b8a172ac    0.032424
Name: 2017-08-08 00:00:00, dtype: float64
1e0c232022c226ff39f67cc3e0686593237394f1989b2c08e76d0a21 {'OEF': 0.43172637776288608, 'EWZ': 0.042571368363804388}
5522786eb28943c781c8bf62503848a2bdf51ba50e612fd13c5a3f9f bafabbb2ddbb6e294ed821e1e1e41116981d49addb415b95798bb67e    0.541906
e1111d514a667ab22d8f48a2734a7dab0be1350f8ce70ca3b8a172ac    0.458094
Name: 2017-08-08 00:00:00, dtype: float64
5522786eb28943c781c8bf62503848a2bdf51ba50e612fd13c5a3f9f {'EWZ': 0.69460395372295913}
6258238755a1b74f251d1455565b494a05fec30eba348b8a17101c42 a762825ed2a3e080b702b24d23121d77b9e6b6fe24af40f5d3187f44    0.542583
b903708f9b21d044698db01d68f6be915739a688f28c591eb57807c3    0

In [53]:
import importlib
importlib.reload(dataAck)

<module 'dataAck' from '/Users/patrickogrady/Documents/CODE/walkforwardTrader/dataAck.py'>

In [16]:
'Adj_Close_VO' in joinedData.columns

True

In [22]:
##UPDATE CACHE
portfolioInfos = []
for portfolioHash in modelsInPortfolio:
    portfolioInfo = portfolio.getPortfolioByKey(portfolioHash)
    portfolioInfo = {
        "key":portfolioInfo.key.name,
        "description":portfolioInfo["description"],
        "benchmark":portfolioInfo["benchmark"]
    }
    print(portfolioInfo)
    portfolioInfos.append(portfolioInfo)

##GET ALL BENCHMARKS
benchmarksNeeded = []
print(joinedData.columns.values)
for info in portfolioInfos:
    print(info["benchmark"])
    if info["benchmark"] not in benchmarksNeeded and ("Adj_Close_" + info["benchmark"] not in joinedData.columns.values):
        benchmarksNeeded.append(info["benchmark"])
print("BENCHMARKS NEEDED", benchmarksNeeded)
pulledData, unused_ = dataAck.downloadTickerData(benchmarksNeeded)
thisJoinedData = joinedData
for ticker in pulledData:
    thisJoinedData = joinedData.join(pulledData[ticker], how='outer')

for info in portfolioInfos:
    portfolioHash = info["key"]
    print(portfolioHash)
    portfolioData = portfolioGeneration.getDataForPortfolio(portfolioHash, portfolioInfo["benchmark"], thisJoinedData)
    portfolioGeneration.cachePortfolio(info, portfolioData)

{'key': '1e0c232022c226ff39f67cc3e0686593237394f1989b2c08e76d0a21', 'description': 'First Combo Trade Test vs SPY', 'benchmark': 'SPY'}
{'key': '5522786eb28943c781c8bf62503848a2bdf51ba50e612fd13c5a3f9f', 'description': 'Just trade underlying', 'benchmark': 'EWZ'}
{'key': '6258238755a1b74f251d1455565b494a05fec30eba348b8a17101c42', 'description': 'Just trade underlying vs SPY', 'benchmark': 'SPY'}
{'key': '7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f', 'description': 'VO 8-13-17', 'benchmark': 'VO'}
['Adj_Close_VO' 'Adj_Close_XLV' 'Adj_Close_EWZ' 'Adj_Close_GDX'
 'Adj_Close_SHY' 'Adj_Close_EWT' 'Adj_Close_IEF' 'Adj_Close_OEF'
 'Adj_Close_TIP' 'Adj_Close_IWF' 'Adj_Close_EWH']
SPY
EWZ
SPY
VO
BENCHMARKS NEEDED ['SPY']
SPY
((22, ('XLV', 26, None, None), 3, 'OEF'), 22, 25)
((10, ('GDX', 32, None, None), 3, 'EWZ'), 10, 50)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((22, ('TIP', 29, None, None), 3, 'OEF'), 22, 25)
((22, ('GDX', 32, None, None), 3, 'EWZ'), 22, 100)
((10, ('G

In [35]:
def getNetAllocationAcrossPortfolios(portfolioHashes):
    totalDesired = {}
    for portfolioHash in portfolioHashes:
        portfolioEntity = portfolio.getPortfolioAllocations(portfolioHash, portfolio.getToday())[0]
        netPosition = {}
        for key in portfolioEntity:
            if key.startswith("ticker_"):
                netPosition[key[len("ticker_"):]] = portfolioEntity[key]
                
        print(portfolioHash, netPosition)
        
        for ticker in netPosition:
            if ticker not in totalDesired:
                totalDesired[ticker] = 0.0
            totalDesired[ticker] += netPosition[ticker]
    totalSum = sum([abs(totalDesired[item]) for item in totalDesired])
    toReturn = {}
    for ticker in totalDesired:
        toReturn[ticker] = totalDesired[ticker]/totalSum
    print("TOTAL", toReturn)
    return toReturn
    

In [36]:
##SEND TO IB
getNetAllocationAcrossPortfolios([portfolioInfo["key"] for portfolioInfo in allPortfolios])

1e0c232022c226ff39f67cc3e0686593237394f1989b2c08e76d0a21 {'OEF': 0.4317263777628861, 'EWZ': 0.04257136836380439}
5522786eb28943c781c8bf62503848a2bdf51ba50e612fd13c5a3f9f {'EWZ': 0.6946039537229591}
6258238755a1b74f251d1455565b494a05fec30eba348b8a17101c42 {'EWT': 0.5425829298660056}
7d372f0c7053ce8035616e2ba82f33de0cfebccbd78e4e3798f0c18f {'VO': 0.3278538656243243}
TOTAL {'OEF': 0.21169922440507488, 'EWZ': 0.36147766727851055, 'EWT': 0.26605829836775147, 'VO': 0.16076480994866305}


{'EWT': 0.26605829836775147,
 'EWZ': 0.36147766727851055,
 'OEF': 0.21169922440507488,
 'VO': 0.16076480994866305}